In [11]:
"""
Kaggle Submission Notebook for CSIRO Biomass Prediction
========================================================
This notebook runs inference using trained models and creates submission.csv

Usage:
1. Train models locally: python train.py --config configs/exp/exp001.yaml
2. Upload model weights and code to Kaggle (run cell-2 locally)
3. Upload this notebook to Kaggle with the data sources attached
4. Run this notebook on Kaggle
"""

import os
import sys

# Kaggle環境ではコードDatasetからインポートするためにsys.pathを設定
# Dataset名: csiro-biomass-codes-{EXP_NAME} (例: csiro-biomass-codes-001)
if os.getenv("KAGGLE_DATA_PROXY_TOKEN"):
    sys.path.insert(0, "/kaggle/input/csiro-biomass-codes-014")

import config
import pandas as pd

# Check environment
print(f"Running in Kaggle: {config.IS_KAGGLE_ENV}")
print(f"Input directory: {config.INPUT_DIR}")
print(f"Output directory: {config.OUTPUT_DIR}")
print(f"Artifact directory: {config.ARTIFACT_DIR}")


Running in Kaggle: False
Input directory: /workspace/data/input
Output directory: /workspace/data/output/014/1
Artifact directory: /workspace/data/output


In [ ]:
# # Run inference (SVR)
# from inference_svr import kaggle_inference

# # Configuration - Change experiment_dir to match your SVR experiment
# EXPERIMENT_DIR = "20251228_XXXXXX_exp014_dinov3_svr"  # <- ここを変更
# FOLDS = None  # None = use all 5 folds (0-4), or specify list like [0, 1, 2]
# IMG_SIZE = 960  # Image size (must be divisible by 16)

# # Run inference and create submission
# # TTA (4 flip variants) is always enabled
# submission_df = kaggle_inference(
#     experiment_dir=EXPERIMENT_DIR,
#     folds=FOLDS,
#     img_size=IMG_SIZE,
# )

# # Save submission
# submission_df.to_csv("submission.csv", index=False)
# print("Submission saved to submission.csv")
# print(f"Submission shape: {submission_df.shape}")
# submission_df.head(10)


In [12]:
# Upload artifacts to Kaggle (run this locally after training)
# Uncomment and run this cell to upload trained models and code

# ============================================================
# アップロードする experiment_dir を指定
# data/output/014/1/ 内のディレクトリを確認して指定してください
# ============================================================
EXPERIMENT_DIR = "20260101_222630_exp014_dinov3_svr"  # ← ここを変更

if not config.IS_KAGGLE_ENV:
    import shutil

    from src.kaggle_utils.customhub import dataset_upload, model_upload

    experiment_path = config.OUTPUT_DIR / EXPERIMENT_DIR
    if not experiment_path.exists():
        raise FileNotFoundError(f"Experiment directory not found: {experiment_path}")

    print(f"Uploading experiment: {experiment_path}")

    # Copy backbone.pth to experiment directory for upload
    # backbone.pth is saved at OUTPUT_DIR/backbone.pth during feature extraction
    backbone_src = config.OUTPUT_DIR / "backbone.pth"
    backbone_dst = experiment_path / "backbone.pth"
    if backbone_src.exists() and not backbone_dst.exists():
        shutil.copy2(backbone_src, backbone_dst)
        print(f"Copied backbone.pth to {backbone_dst}")
    elif backbone_dst.exists():
        print(f"backbone.pth already exists at {backbone_dst}")
    else:
        print(f"WARNING: backbone.pth not found at {backbone_src}")

    # モデル重みを Kaggle Model としてアップロード
    # experiment_dir 内の全 run_name をアップロード
    model_upload(
        handle=config.ARTIFACTS_HANDLE,
        local_model_dir=experiment_path,
        update=True,  # Set to True to update existing model
    )

    # 実験コードを Kaggle Dataset としてアップロード
    # experiments/014/ ディレクトリのみ（プロジェクト全体ではない）
    dataset_upload(
        handle=config.CODES_HANDLE,
        local_dataset_dir=config.EXP_DIR,  # experiments/014/ のみ
        update=True,
    )


Uploading experiment: /workspace/data/output/014/1/20260101_222630_exp014_dinov3_svr
backbone.pth already exists at /workspace/data/output/014/1/20260101_222630_exp014_dinov3_svr/backbone.pth
dst_dir=/tmp/tmpybnzw7oa
tree
├── 001
│   ├── config.yaml
│   ├── fold_results.json
│   ├── oof_predictions.csv
│   ├── train.log
│   └── weights
│       ├── svr_fold0.pkl
│       ├── svr_fold1.pkl
│       ├── svr_fold2.pkl
│       ├── svr_fold3.pkl
│       └── svr_fold4.pkl
└── backbone.pth
Starting upload for file 001.zip


100%|██████████| 44.5M/44.5M [00:03<00:00, 13.7MB/s]


Upload successful: 001.zip (44MB)
Starting upload for file backbone.pth


100%|██████████| 3.13G/3.13G [02:34<00:00, 21.8MB/s]


Upload successful: backbone.pth (3GB)
dst_dir=/tmp/tmp3mqf7h_1/csiro-biomass-codes-014
tree
├── README.md
├── code.ipynb
├── config.py
├── configs
│   └── exp
│       └── exp014_dinov3_svr.yaml
├── extract_features.py
├── inference_svr.py
├── src
│   ├── __init__.py
│   ├── backbone.py
│   ├── data.py
│   ├── kaggle_utils
│   │   ├── __init__.py
│   │   └── customhub.py
│   ├── metric.py
│   ├── seed.py
│   └── svr_model.py
├── train_svr.py
└── train_svr_update_cv.py
Starting upload for file code.ipynb


...

Upload successful: README.md (0B)
Starting upload for file configs.zip


100%|██████████| 671/671 [00:00<00:00, 862B/s]


Upload successful: configs.zip (671B)
Starting upload for file config.py


100%|██████████| 2.88k/2.88k [00:00<00:00, 3.77kB/s]


Upload successful: config.py (3KB)
